In [14]:
import pandas as pd
import json
import os

import json
import requests

import datetime
import math


#----- API request-----
url = 'https://api.windy.com/api/point-forecast/v2'  #windy point API
#内灘町の緯度経度　とりあえずのKEY
data = {
    "lat": xx.xxx,  
    "lon": yyy.yyy,
    "model": "gfs",
    "parameters": ["wind","temp","ptype","windGust","gh"],
    "levels": ["surface", "1000h", "950h"],
    "key": "your API key"
}
headers = {
    'Content-Type': 'application/json',
}
r = requests.post('https://api.windy.com/api/point-forecast/v2', json = data)
#dict型のJSONなので数合わせしてから
r = json.loads(r.text)

df=pd.DataFrame.from_dict({
    'ts':r['ts'],
    'wind_u_0':r['wind_u-surface'],
    'wind_v_0':r['wind_v-surface'],
    'wind_u_100':r['wind_u-1000h'],
    'wind_v_100':r['wind_v-1000h'],
    'wind_u_750':r['wind_u-950h'],
    'wind_v_750':r['wind_v-950h'],
    'wind_max':r['gust-surface'],
    'temp':r['temp-surface'],
    'ptype-surface':r['ptype-surface']
})

dt=[]
dwind_ms0=[]
dwind_d0=[]
dwind_ms100=[]
dwind_d100=[]
dwind_ms750=[]
dwind_d750=[]

for i,ts in enumerate(df['ts']):
    if df['wind_u_0'][i]==0 : df['wind_u_0'][i]=0.000001
    if df['wind_v_0'][i]==0 : df['wind_v_0'][i]=0.000001
    if df['wind_u_100'][i]==0 : df['wind_u_100'][i]=0.000001
    if df['wind_v_100'][i]==0 : df['wind_v_100'][i]=0.000001
    if df['wind_u_750'][i]==0 : df['wind_u_750'][i]=0.000001
    if df['wind_v_750'][i]==0 : df['wind_v_750'][i]=0.000001
    dt.append(datetime.datetime.fromtimestamp(ts/1000))
    dwind_ms0.append(math.sqrt(df['wind_u_0'][i]**2+df['wind_v_0'][i]**2))
    dwind_d0.append(90-math.atan(df['wind_v_0'][i]/df['wind_u_0'][i])/math.pi*180)
    dwind_ms100.append(math.sqrt(df['wind_u_100'][i]**2+df['wind_v_100'][i]**2))
    dwind_d100.append(90-math.atan(df['wind_v_100'][i]/df['wind_u_100'][i])/math.pi*180)
    dwind_ms750.append(math.sqrt(df['wind_u_750'][i]**2+df['wind_v_750'][i]**2))
    dwind_d750.append(90-math.atan(df['wind_v_750'][i]/df['wind_u_750'][i])/math.pi*180)

df['TimeS']=dt               #Timestamp型に変更
df['temp']=df['temp']-273.15 #K ->℃
df['wind_ms0']=dwind_ms0     # m/s 
df['wind_d0']=dwind_d0       #風向き角度に変更
df['wind_ms100']=dwind_ms100
df['dwind_d100']=dwind_d100
df['wind_ms750']=dwind_ms750
df['wind_d750']=dwind_d750


fname=str(df['TimeS'][0].strftime("%Y%m%d%M%I%S"))+"WINDY.csv"
df.to_csv(fname)

'20220113000600'